In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
data = pd.read_csv('Heart Dataset.csv', header=None)

In [3]:
# Set column names
data.columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

In [4]:
# Replace missing values with NaN
data = data.replace({'?': np.nan})

In [5]:
# Convert to numerical type
data = data.apply(pd.to_numeric, errors='coerce')

In [6]:
# Replace missing values with the average of the corresponding column
imp = SimpleImputer(strategy='mean')
data = pd.DataFrame(imp.fit_transform(data), columns=data.columns)

In [7]:
# Convert categorical variables to binary indicator variables
data = pd.get_dummies(data, columns=['cp', 'restecg', 'slope', 'thal'])

In [8]:
# Split data into features and target
X = data.drop(['target'], axis=1).values
y = data['target'].values

In [9]:
# Scale data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [10]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Reshape data for LSTM input
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [12]:
# Define model architecture
model = tf.keras.Sequential([
    layers.LSTM(64, input_shape=(X_train.shape[1], 1)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [13]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
8/8 [==============================] - 7s 149ms/step - loss: 0.5704 - accuracy: 0.4202 - val_loss: 0.6599 - val_accuracy: 0.1667
Epoch 2/50
8/8 [==============================] - 0s 22ms/step - loss: -0.0845 - accuracy: 0.4622 - val_loss: 0.6124 - val_accuracy: 0.1667
Epoch 3/50
8/8 [==============================] - 0s 20ms/step - loss: -0.6020 - accuracy: 0.5084 - val_loss: 0.5485 - val_accuracy: 0.1667
Epoch 4/50
8/8 [==============================] - 0s 19ms/step - loss: -0.8272 - accuracy: 0.5126 - val_loss: 0.4680 - val_accuracy: 0.1667
Epoch 5/50
8/8 [==============================] - 0s 19ms/step - loss: -1.3014 - accuracy: 0.4958 - val_loss: 0.4119 - val_accuracy: 0.1667
Epoch 6/50
8/8 [==============================] - 0s 18ms/step - loss: -1.3973 - accuracy: 0.5336 - val_loss: 0.3698 - val_accuracy: 0.1667
Epoch 7/50
8/8 [==============================] - 0s 20ms/step - loss: -1.6602 - accuracy: 0.5168 - val_loss: 0.3550 - val_accuracy: 0.1667
Epoch 8/50
8/8 [====

In [30]:
# Make predictions on new data
new_data = np.array([[63, 1, 1, 145, 233, 1, 2, 150, 0, 2.3, 3, 0, 6]])

In [31]:
# Add 9 zeros to each input array
new_data = np.hstack((new_data, np.zeros((len(new_data), 9))))

In [32]:
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = np.reshape(new_data_scaled, (new_data_scaled.shape[0], new_data_scaled.shape[1], 1))
predictions = model.predict(new_data_reshaped)

1/1 [==============================] - 0s 25ms/step


In [33]:
#This statement will print the predicted values for the input data provided to the model. The output will be an array of 
#predicted values for each input sample, where each value represents the probability of having a heart disease. If the value is 
#closer to 1, it indicates a higher probability of having a heart disease, and if it's closer to 0, it indicates a lower 
#probability.
print(predictions)

[[1.]]
